In [1]:
import MakingSequences
import makingPairwiseDistancesOurMethod
import makingFinalReconstructedTreeOurMethod
import makingOtherMethodTree
import comparingTrees
import time

In [2]:
seqLenInput = 1e7
initial_size_Var=100
# time_Var = 200_000
mu_Var = 1e-8
recombination_rate=1e-8

In [3]:
birth_rate = 1000.0
death_rate = 500.0
n_populations = 5
extra_populations = 5 # for "GSA" sampling scheme

In [6]:
st = time.time()
seqs, tree = MakingSequences.makeSequencesFromScratch(birth_rate, death_rate, n_populations, extra_populations, seqLenInput, initial_size_Var, mu_Var, recombination_rate)
print((time.time()-st)/60)

Demography
╟  Populations
║  ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
║  │ id │name   │description  │initial_size  │ growth_rate │  default_sampling_time│extra_metadata  │
║  ├────────────────────────────────────────────────────────────────────────────────────────────────┤
║  │ 0  │T3     │             │100.0         │      0      │                      0│{}              │
║  │ 1  │T2     │             │100.0         │      0      │                      0│{}              │
║  │ 2  │T1     │             │100.0         │      0      │                      0│{}              │
║  │ 3  │T5     │             │100.0         │      0      │                      0│{}              │
║  │ 4  │T4     │             │100.0         │      0      │                      0│{}              │
║  │ 5  │pop_5  │             │100.0         │      0      │                2.7e+04│{}              │
║  │ 6  │pop_6  │             │100.0         │      0   

In [ ]:
st = time.time()
mats = makingFinalReconstructedTreeOurMethod.makeManyMatricesOfDistances(10, seqs)
print((time.time()-st)/60)

in getting kmer vectors, working on k: 9 seq 0
in getting kmer vectors, working on k: 9 seq 1
in getting kmer vectors, working on k: 9 seq 2
in getting kmer vectors, working on k: 9 seq 3
in getting kmer vectors, working on k: 9 seq 4
in getting kmer vectors, working on k: 10 seq 0
in getting kmer vectors, working on k: 10 seq 1
in getting kmer vectors, working on k: 10 seq 2
in getting kmer vectors, working on k: 10 seq 3
in getting kmer vectors, working on k: 10 seq 4
in getting kmer vectors, working on k: 11 seq 0
in getting kmer vectors, working on k: 11 seq 1
in getting kmer vectors, working on k: 11 seq 2
in getting kmer vectors, working on k: 11 seq 3
in getting kmer vectors, working on k: 11 seq 4
in getting kmer vectors, working on k: 12 seq 0


In [ ]:
finalDistMat = makingFinalReconstructedTreeOurMethod.makeFinalDistMatrixFromSmallerOnes(mats)

In [11]:
import numpy as np
finalDistMat
average = np.average(finalDistMat)
for i in range(len(finalDistMat[0])):
    for j in range(len(finalDistMat[0])):
        if i != j:
            if finalDistMat[i][j] == 0.0:
                finalDistMat[i][j] += average
finalDistMat

,0,1,2,3,4
0,0.000000,0.000251,0.000441,0.000396,0.000803
1,0.000251,0.000000,0.000377,0.000444,0.000564
2,0.000441,0.000377,0.000000,0.000370,0.000660
3,0.000396,0.000444,0.000370,0.000000,0.000693
4,0.000803,0.000564,0.000660,0.000693,0.000000


In [26]:
finalDistMat*2

,0,1,2,3,4
0,0.000000,0.000501,0.000883,0.000791,0.001605
1,0.000501,0.000000,0.000755,0.000888,0.001129
2,0.000883,0.000755,0.000000,0.000741,0.001320
3,0.000791,0.000888,0.000741,0.000000,0.001386
4,0.001605,0.001129,0.001320,0.001386,0.000000


In [23]:
our_tree = makingFinalReconstructedTreeOurMethod.makeTree(seqs, mu_Var , (finalDistMat*2))


print(our_tree.as_string("newick"))
our_tree.print_plot()

[&U] (T5:17426.554218113255,(T1:39211.368914145874,(T3:94537.48878611736,(T4:33522.15481820746,T2:16610.455748307362):17108.361388777346):3713.661514224135):17426.554218113255);

/--------------------------------------------------------------------------- T5
+                                                                              
|                  /-------------------------------------------------------- T1
\------------------+                                                           
                   |                  /------------------------------------- T3
                   \------------------+                                        
                                      |                  /------------------ T4
                                      \------------------+                     
                                                         \------------------ T2
                                                                               
                     

In [15]:
theirTree = makingOtherMethodTree.inferTreeFromSeqs(seqs, mu_Var)

T4	0
T2	0.0006837750000000531	0
T5	0.0008627749999999823	0.0008663249999999456	0
T1	0.0008588750000000367	0.0008622499999999533	0.0001875000000000071	0
T3	0.0015962499999999658	0.0015998499999999582	0.0016019750000000332	0.001597900000000041	0
	T4	T2	T5	T1	T3


In [16]:
theirTree.print_plot()
print(str(theirTree))

                                                  /------------------------- T2
/-------------------------------------------------+                            
|                                                 \------------------------- T4
+                                                                              
|                        /-------------------------------------------------- T3
\------------------------+                                                     
                         |                        /------------------------- T5
                         \------------------------+                            
                                                  \------------------------- T1
                                                                               
                                                                               
((T2:34365.31250000097,T4:34012.187500004344):4422.343749996283,(T3:116771.56250000097,(T5:9575.833333332524,T1:9174.166

In [13]:
print(str(tree))
tree.print_plot()

(((T4:33893.09336373573,T2:33893.09336373573):8952.776937659539,(T5:9332.09193210266,T1:9332.09193210266):33513.77836929261):37304.89560273798,T3:80150.76590413325):117282.94688582307
                                                  /------------------------- T4
                         /------------------------+                            
                         |                        \------------------------- T2
/------------------------+                                                     
|                        |                        /------------------------- T5
+                        \------------------------+                            
|                                                 \------------------------- T1
|                                                                              
\--------------------------------------------------------------------------- T3
                                                                               
                

In [17]:
print('==== REAL')
pdc = tree.phylogenetic_distance_matrix()
for i, t1 in enumerate(tree.taxon_namespace[:-1]):
    for t2 in tree.taxon_namespace[i+1:]:
        print("Distance between '%s' and '%s': %s" % (t1.label, t2.label, pdc(t1, t2)))
print('====')
print('==== THEM')
pdc = theirTree.phylogenetic_distance_matrix()
for i, t1 in enumerate(theirTree.taxon_namespace[:-1]):
    for t2 in theirTree.taxon_namespace[i+1:]:
        print("Distance between '%s' and '%s': %s" % (t1.label, t2.label, pdc(t1, t2)))
print('====')
print('==== US')
pdc = our_tree.phylogenetic_distance_matrix()
for i, t1 in enumerate(our_tree.taxon_namespace[:-1]):
    for t2 in our_tree.taxon_namespace[i+1:]:
        print("Distance between '%s' and '%s': %s" % (t1.label, t2.label, pdc(t1, t2)))
print('====')

==== REAL
Distance between 'T1' and 'T2': 85691.74060279054
Distance between 'T1' and 'T3': 160301.5318082665
Distance between 'T1' and 'T4': 85691.74060279054
Distance between 'T1' and 'T5': 18664.18386420532
Distance between 'T2' and 'T3': 160301.5318082665
Distance between 'T2' and 'T4': 67786.18672747146
Distance between 'T2' and 'T5': 85691.74060279054
Distance between 'T3' and 'T4': 160301.5318082665
Distance between 'T3' and 'T5': 160301.5318082665
Distance between 'T4' and 'T5': 85691.74060279054
====
==== THEM
Distance between 'T4' and 'T2': 68377.50000000531
Distance between 'T4' and 'T5': 86279.89583333182
Distance between 'T4' and 'T1': 85878.22916666747
Distance between 'T4' and 'T3': 159628.43749999788
Distance between 'T2' and 'T5': 86633.02083332844
Distance between 'T2' and 'T1': 86231.3541666641
Distance between 'T2' and 'T3': 159981.5624999945
Distance between 'T5' and 'T1': 18750.000000000713
Distance between 'T5' and 'T3': 160194.58333333588
Distance between 'T1' a

In [162]:

for edge in our_tree.postorder_edge_iter():
    if edge.length != None:
        edge.length /= 200

In [163]:
comparingTrees.computeDistancesOfTrees(theirTree, tree, n_populations)

7.0 1.75 165807.60534179208 148990.15906985686 1.4964587022336173


(7.0, 1.75, 165807.60534179208, 148990.15906985686, 1.4964587022336173)

In [164]:
comparingTrees.computeDistancesOfTrees(tree, our_tree, n_populations)

7.0 1.75 143447.72056300173 117265.09262800806 1.2036716063815682


(7.0, 1.75, 143447.72056300173, 117265.09262800806, 1.2036716063815682)